<a href="https://colab.research.google.com/github/lizehrhardt/recaptcha/blob/main/captcha_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/captcha/large/train"
!pwd

 Bicycle   Bus	 Chimney     Hydrant	  Mountain   Palm    'Traffic Light'
 Bridge    Car	 Crosswalk   Motorcycle   Other      Stairs
/content


In [ ]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

large_labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Mountain', 'Other', 'Palm', 'Stairs', 'Traffic Light']
small_labels = ['Car', 'Crosswalk', 'Hydrant', 'TrafficLight']

In [ ]:
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

image_size = 128
batch_size = 32

np_data = 'numpy_data/'

large_train_path = "/content/drive/MyDrive/captcha/large/train"
large_test_path = "/content/drive/MyDrive/captcha/large/val"

small_path = "/content/drive/MyDrive/captcha/small/"

In [ ]:
def save_data(data_dir, file_prefix, labels):
    data = [] 
    if (not os.path.exists(data_dir)):
      os.mkdir(data_dir)
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                #img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                img_path = os.path.join(path, img)
                img = keras.preprocessing.image.load_img(img_path, target_size=(image_size, image_size))
                img_arr = keras.preprocessing.image.img_to_array(img)
                img_arr = img_arr/255.0
                #img_arr = cv2.imread(os.path.join(path,img),1)
                #resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                #data.append([resized_arr, class_num])
                data.append([img_arr, class_num])
            except Exception as e:
                print(e)
        np.save(np_data+file_prefix+label, data)
    return np.array(data)

In [ ]:
def get_large_data(data_dir):
    data = [] 
    for label in large_labels: 
        path = os.path.join(data_dir, label)
        class_num = large_labels.index(label)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img = keras.preprocessing.image.load_img(img_path, target_size=(image_size, image_size))
                img_arr = keras.preprocessing.image.img_to_array(img)
                img_arr = img_arr/255.0
                #resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                #data.append([resized_arr, class_num])
                data.append([img_arr, class_num])
            except Exception as e:
                print(e)
        np.save('numpy_data/Large_'+label, data)
    return np.array(data)

In [ ]:
small_data = save_data(small_path,'Small_',small_labels)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
large_train_data = save_data(large_train_path,'Large_Train_',large_labels)
large_test_data = save_data(large_test_path,'Large_Test_',large_labels)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
base_model = tf.keras.applications.MobileNet(input_shape=(image_size,image_size,3), include_top=False)

base_model.trainable = False

base_model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(13, activation='sigmoid')
])

base_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

base_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 13)                13325     
                                                                 
Total params: 3,242,189
Trainable params: 13,325
Non-trainable params: 3,228,864
_________________________________________________________________


In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(directory=large_train_path, target_size=(image_size, image_size))

Found 16439 images belonging to 13 classes.


In [ ]:
validation_datagen = keras.preprocessing.image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(directory=large_test_path, target_size=(image_size, image_size))

Found 4117 images belonging to 13 classes.


In [ ]:
epochs = 25

steps_per_epoch = np.ceil(train_generator.n / batch_size)
validation_steps = np.ceil(validation_generator.n / batch_size)

history = base_model.fit_generator(generator=train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

base_model.save('MobileNet_TransferLearning_Captcha.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/25
514/514 [==============================] - 281s 540ms/step - loss: 0.2789 - accuracy: 0.2468 - val_loss: 0.2200 - val_accuracy: 0.3245
Epoch 2/25
514/514 [==============================] - 270s 525ms/step - loss: 0.2114 - accuracy: 0.3627 - val_loss: 0.2055 - val_accuracy: 0.3811
Epoch 3/25
514/514 [==============================] - 276s 536ms/step - loss: 0.2011 - accuracy: 0.3993 - val_loss: 0.1981 - val_accuracy: 0.4088
Epoch 4/25
514/514 [==============================] - 271s 528ms/step - loss: 0.1952 - accuracy: 0.4224 - val_loss: 0.1936 - val_accuracy: 0.4265
Epoch 5/25
514/514 [==============================] - 273s 532ms/step - loss: 0.1912 - accuracy: 0.4366 - val_loss: 0.1907 - val_accuracy: 0.4336
Epoch 6/25
514/514 [==============================] - 272s 529ms/step - loss: 0.1882 - accuracy: 0.4482 - val_loss: 0.1882 - val_accuracy: 0.4515
Epoch 7/25
514/514 [==============================] - 269s 524ms/step - loss: 0.1859 - accuracy: 0.4571 - val_loss: 0.1864 -